# Insurance Layer Optimization

Interactive analysis and optimization of multi-layer insurance programs.

In [1]:
import sys
from pathlib import Path

# Add parent directory to path
notebook_dir = Path().absolute()
parent_dir = notebook_dir.parent
sys.path.insert(0, str(parent_dir))

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, HTML

from src.insurance_program import (
    EnhancedInsuranceLayer,
    InsuranceProgram
)
from src.loss_distributions import (
    ManufacturingLossGenerator,
    LossEvent
)
from src.visualization import (
    WSJ_COLORS,
    format_currency
)

# Set default plotly theme
import plotly.io as pio
pio.templates.default = "plotly_white"

print("Insurance Layer Optimization Notebook")
print("="*50)

Insurance Layer Optimization Notebook


## 1. Interactive Layer Structure Design

In [2]:
# Interactive widgets for layer design
layer1_attachment = widgets.IntSlider(
    value=0, min=0, max=1000000, step=100000,
    description='L1 Attach:', continuous_update=False
)
layer1_limit = widgets.IntSlider(
    value=5000000, min=1000000, max=10000000, step=500000,
    description='L1 Limit:', continuous_update=False
)
layer1_premium = widgets.FloatSlider(
    value=0.015, min=0.005, max=0.05, step=0.001,
    description='L1 Rate:', continuous_update=False
)

layer2_attachment = widgets.IntSlider(
    value=5000000, min=1000000, max=10000000, step=500000,
    description='L2 Attach:', continuous_update=False
)
layer2_limit = widgets.IntSlider(
    value=20000000, min=5000000, max=50000000, step=5000000,
    description='L2 Limit:', continuous_update=False
)
layer2_premium = widgets.FloatSlider(
    value=0.008, min=0.002, max=0.03, step=0.001,
    description='L2 Rate:', continuous_update=False
)

layer3_attachment = widgets.IntSlider(
    value=25000000, min=10000000, max=50000000, step=5000000,
    description='L3 Attach:', continuous_update=False
)
layer3_limit = widgets.IntSlider(
    value=25000000, min=10000000, max=100000000, step=10000000,
    description='L3 Limit:', continuous_update=False
)
layer3_premium = widgets.FloatSlider(
    value=0.004, min=0.001, max=0.02, step=0.001,
    description='L3 Rate:', continuous_update=False
)

n_simulations = widgets.IntSlider(
    value=1000, min=100, max=10000, step=100,
    description='Simulations:', continuous_update=False
)

def analyze_layer_structure(l1_attach, l1_limit, l1_rate,
                           l2_attach, l2_limit, l2_rate,
                           l3_attach, l3_limit, l3_rate,
                           n_sims):
    """Analyze insurance layer structure."""
    
    # Create insurance program
    layers = [
        EnhancedInsuranceLayer(l1_attach, l1_limit, l1_rate),
        EnhancedInsuranceLayer(l2_attach, l2_limit, l2_rate),
        EnhancedInsuranceLayer(l3_attach, l3_limit, l3_rate)
    ]
    program = InsuranceProgram(layers)
    
    # Create loss generator
    loss_generator = ManufacturingLossGenerator(
        attritional_params={
            'base_frequency': 5.0,
            'severity_mean': 50_000,
            'severity_cv': 0.8
        },
        large_params={
            'base_frequency': 0.5,
            'severity_mean': 2_000_000,
            'severity_cv': 1.2
        },
        catastrophic_params={
            'base_frequency': 0.02,
            'severity_xm': 10_000_000,
            'severity_alpha': 2.5
        },
        seed=42
    )
    
    # Simulate losses and recoveries
    results = []
    layer_utilization = {i: [] for i in range(3)}
    
    for _ in range(n_sims):
        events, stats = loss_generator.generate_losses(duration=1.0, revenue=10_000_000)
        
        total_loss = 0
        total_recovery = 0
        layer_recoveries = [0, 0, 0]
        
        for event in events:
            loss_amount = event.amount
            total_loss += loss_amount
            
            # Process through layers
            recovery_details = program.process_claim(loss_amount)
            recovery_amount = recovery_details['insurance_recovery']
            total_recovery += recovery_amount
            
            # Track layer utilization from the layers_triggered list
            for layer_info in recovery_details['layers_triggered']:
                layer_idx = layer_info['layer_index']
                layer_recoveries[layer_idx] += layer_info['payment']
        
        results.append({
            'total_loss': total_loss,
            'total_recovery': total_recovery,
            'retained': total_loss - total_recovery,
            'layer1_recovery': layer_recoveries[0],
            'layer2_recovery': layer_recoveries[1],
            'layer3_recovery': layer_recoveries[2]
        })
        
        for i in range(3):
            layer_utilization[i].append(layer_recoveries[i])
    
    df = pd.DataFrame(results)
    
    # Create visualizations
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Layer Structure Visualization',
            'Recovery Distribution',
            'Layer Utilization',
            'Retention Analysis'
        ),
        specs=[
            [{'type': 'bar'}, {'type': 'histogram'}],
            [{'type': 'box'}, {'type': 'scatter'}]
        ]
    )
    
    # Layer structure visualization
    layer_data = [
        {'Layer': 'Layer 1', 'Start': l1_attach, 'Size': l1_limit, 'End': l1_attach + l1_limit},
        {'Layer': 'Layer 2', 'Start': l2_attach, 'Size': l2_limit, 'End': l2_attach + l2_limit},
        {'Layer': 'Layer 3', 'Start': l3_attach, 'Size': l3_limit, 'End': l3_attach + l3_limit}
    ]
    
    for i, ld in enumerate(layer_data):
        fig.add_trace(
            go.Bar(
                x=[ld['Size']],
                y=[ld['Layer']],
                orientation='h',
                name=ld['Layer'],
                marker_color=[WSJ_COLORS['blue'], WSJ_COLORS['orange'], WSJ_COLORS['green']][i],
                base=[ld['Start']]
            ),
            row=1, col=1
        )
    
    # Recovery distribution
    fig.add_trace(
        go.Histogram(
            x=df['total_recovery'],
            nbinsx=30,
            name='Recovery',
            marker_color=WSJ_COLORS['blue']
        ),
        row=1, col=2
    )
    
    # Layer utilization box plots
    for i in range(3):
        fig.add_trace(
            go.Box(
                y=layer_utilization[i],
                name=f'Layer {i+1}',
                marker_color=[WSJ_COLORS['blue'], WSJ_COLORS['orange'], WSJ_COLORS['green']][i]
            ),
            row=2, col=1
        )
    
    # Retention vs Loss scatter
    fig.add_trace(
        go.Scatter(
            x=df['total_loss'],
            y=df['retained'],
            mode='markers',
            marker=dict(
                size=5,
                color=df['total_recovery'],
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(title="Recovery")
            ),
            name='Scenarios'
        ),
        row=2, col=2
    )
    
    # Update layout
    fig.update_layout(
        height=800,
        showlegend=False,
        title_text="Insurance Layer Structure Analysis",
        template='plotly_white'
    )
    
    fig.update_xaxes(title_text="Coverage Amount", row=1, col=1, tickformat='$.2s')
    fig.update_xaxes(title_text="Recovery Amount", row=1, col=2, tickformat='$.2s')
    fig.update_xaxes(title_text="Layer", row=2, col=1)
    fig.update_xaxes(title_text="Total Loss", row=2, col=2, tickformat='$.2s')
    
    fig.update_yaxes(title_text="Layer", row=1, col=1)
    fig.update_yaxes(title_text="Frequency", row=1, col=2)
    fig.update_yaxes(title_text="Recovery Amount", row=2, col=1, tickformat='$.2s')
    fig.update_yaxes(title_text="Retained Loss", row=2, col=2, tickformat='$.2s')
    
    fig.show()
    
    # Calculate and display metrics
    total_premium = program.calculate_annual_premium()
    avg_recovery = df['total_recovery'].mean()
    avg_retained = df['retained'].mean()
    
    # Layer utilization stats
    layer_stats = []
    for i in range(3):
        util = layer_utilization[i]
        layer_stats.append({
            'Layer': f'Layer {i+1}',
            'Attachment': [l1_attach, l2_attach, l3_attach][i],
            'Limit': [l1_limit, l2_limit, l3_limit][i],
            'Rate': [l1_rate, l2_rate, l3_rate][i],
            'Premium': [l1_limit * l1_rate, l2_limit * l2_rate, l3_limit * l3_rate][i],
            'Avg Recovery': np.mean(util),
            'Max Recovery': np.max(util),
            'Utilization %': 100 * (np.array(util) > 0).mean()
        })
    
    stats_df = pd.DataFrame(layer_stats)
    
    print("\nInsurance Program Summary:")
    print("="*70)
    print(f"Total Annual Premium: ${total_premium:,.0f}")
    print(f"Average Annual Recovery: ${avg_recovery:,.0f}")
    print(f"Average Annual Retention: ${avg_retained:,.0f}")
    print(f"Loss Ratio: {100*avg_recovery/total_premium:.1f}%")
    print("\nLayer Details:")
    print(stats_df.to_string(index=False))

# Create interactive interface
print("Design your insurance layer structure:")
print()

layer1_box = widgets.VBox([
    widgets.HTML("<b>Primary Layer</b>"),
    layer1_attachment,
    layer1_limit,
    layer1_premium
])

layer2_box = widgets.VBox([
    widgets.HTML("<b>Excess Layer 1</b>"),
    layer2_attachment,
    layer2_limit,
    layer2_premium
])

layer3_box = widgets.VBox([
    widgets.HTML("<b>Excess Layer 2</b>"),
    layer3_attachment,
    layer3_limit,
    layer3_premium
])

layers_box = widgets.HBox([layer1_box, layer2_box, layer3_box])
controls = widgets.VBox([layers_box, n_simulations])

output = widgets.interactive_output(
    analyze_layer_structure,
    {
        'l1_attach': layer1_attachment,
        'l1_limit': layer1_limit,
        'l1_rate': layer1_premium,
        'l2_attach': layer2_attachment,
        'l2_limit': layer2_limit,
        'l2_rate': layer2_premium,
        'l3_attach': layer3_attachment,
        'l3_limit': layer3_limit,
        'l3_rate': layer3_premium,
        'n_sims': n_simulations
    }
)

display(controls, output)

Design your insurance layer structure:



Output()

## 2. Layer Optimization Analysis

In [3]:
def optimize_layer_attachments(target_retention_ratio=0.2, n_scenarios=1000):
    """Find optimal layer attachment points."""
    
    # Generate loss scenarios
    loss_generator = ManufacturingLossGenerator(
        attritional_params={
            'base_frequency': 5.0,
            'severity_mean': 50_000,
            'severity_cv': 0.8
        },
        large_params={
            'base_frequency': 0.5,
            'severity_mean': 2_000_000,
            'severity_cv': 1.2
        },
        catastrophic_params={
            'base_frequency': 0.02,
            'severity_xm': 10_000_000,
            'severity_alpha': 2.5
        },
        seed=42
    )
    
    annual_losses = []
    for _ in range(n_scenarios):
        events, stats = loss_generator.generate_losses(duration=1.0, revenue=10_000_000)
        annual_losses.append(stats['total_amount'])
    
    annual_losses = np.array(annual_losses)
    
    # Calculate percentiles for attachment points
    percentiles = [50, 75, 90, 95, 99, 99.5]
    attachment_points = {}
    
    for p in percentiles:
        attachment_points[f'{p}th percentile'] = np.percentile(annual_losses, p)
    
    # Test different layer configurations
    configurations = [
        {'name': 'Conservative', 'attachments': [0, 1_000_000, 5_000_000], 
         'limits': [1_000_000, 4_000_000, 20_000_000],
         'rates': [0.025, 0.015, 0.008]},
        {'name': 'Balanced', 'attachments': [0, 5_000_000, 25_000_000],
         'limits': [5_000_000, 20_000_000, 25_000_000],
         'rates': [0.015, 0.008, 0.004]},
        {'name': 'Aggressive', 'attachments': [1_000_000, 10_000_000, 50_000_000],
         'limits': [9_000_000, 40_000_000, 50_000_000],
         'rates': [0.012, 0.006, 0.002]},
        {'name': 'Optimal', 'attachments': [0, attachment_points['90th percentile'], attachment_points['99th percentile']],
         'limits': [attachment_points['90th percentile'], 
                   attachment_points['99th percentile'] - attachment_points['90th percentile'],
                   attachment_points['99.5th percentile'] - attachment_points['99th percentile']],
         'rates': [0.018, 0.009, 0.003]}
    ]
    
    results = []
    
    for config in configurations:
        layers = [
            EnhancedInsuranceLayer(config['attachments'][i], config['limits'][i], config['rates'][i])
            for i in range(3)
        ]
        program = InsuranceProgram(layers)
        
        total_premium = program.calculate_annual_premium()
        recoveries = []
        retentions = []
        
        for loss in annual_losses:
            recovery_details = program.process_claim(loss)
            total_recovery = recovery_details['insurance_recovery']
            recoveries.append(total_recovery)
            retentions.append(loss - total_recovery)
        
        results.append({
            'Configuration': config['name'],
            'Premium': total_premium,
            'Avg Recovery': np.mean(recoveries),
            'Avg Retention': np.mean(retentions),
            'Max Retention': np.max(retentions),
            '95% VaR Retention': np.percentile(retentions, 95),
            'Loss Ratio': np.mean(recoveries) / total_premium if total_premium > 0 else 0,
            'Efficiency': np.mean(recoveries) / np.mean(annual_losses)
        })
    
    results_df = pd.DataFrame(results)
    
    # Create comparison visualization
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Premium vs Recovery',
            'Retention Distribution',
            'Efficiency Comparison',
            'Configuration Details'
        ),
        specs=[
            [{'type': 'scatter'}, {'type': 'box'}],
            [{'type': 'bar'}, {'type': 'table'}]
        ]
    )
    
    # Premium vs Recovery
    fig.add_trace(
        go.Scatter(
            x=results_df['Premium'],
            y=results_df['Avg Recovery'],
            mode='markers+text',
            text=results_df['Configuration'],
            textposition='top center',
            marker=dict(size=12, color=WSJ_COLORS['blue'])
        ),
        row=1, col=1
    )
    
    # Retention distribution
    for config in configurations:
        layers = [
            EnhancedInsuranceLayer(config['attachments'][i], config['limits'][i], config['rates'][i])
            for i in range(3)
        ]
        program = InsuranceProgram(layers)
        retentions = []
        for loss in annual_losses:
            recovery_details = program.process_claim(loss)
            total_recovery = recovery_details['insurance_recovery']
            retentions.append(loss - total_recovery)
        
        fig.add_trace(
            go.Box(
                y=retentions,
                name=config['name'],
                showlegend=False
            ),
            row=1, col=2
        )
    
    # Efficiency comparison
    fig.add_trace(
        go.Bar(
            x=results_df['Configuration'],
            y=results_df['Efficiency'] * 100,
            marker_color=[WSJ_COLORS['blue'], WSJ_COLORS['orange'], 
                         WSJ_COLORS['green'], WSJ_COLORS['red']]
        ),
        row=2, col=1
    )
    
    # Configuration details table
    fig.add_trace(
        go.Table(
            header=dict(
                values=['Config', 'Premium', 'Loss Ratio', 'Efficiency'],
                fill_color=WSJ_COLORS['light_gray'],
                align='left'
            ),
            cells=dict(
                values=[
                    results_df['Configuration'],
                    ['${:,.0f}'.format(x) for x in results_df['Premium']],
                    ['{:.1f}%'.format(x*100) for x in results_df['Loss Ratio']],
                    ['{:.1f}%'.format(x*100) for x in results_df['Efficiency']]
                ],
                align='left'
            )
        ),
        row=2, col=2
    )
    
    # Update layout
    fig.update_layout(
        height=800,
        showlegend=False,
        title_text=f"Layer Configuration Optimization ({n_scenarios} scenarios)",
        template='plotly_white'
    )
    
    fig.update_xaxes(title_text="Annual Premium", row=1, col=1, tickformat='$.2s')
    fig.update_xaxes(title_text="Configuration", row=1, col=2)
    fig.update_xaxes(title_text="Configuration", row=2, col=1)
    
    fig.update_yaxes(title_text="Avg Recovery", row=1, col=1, tickformat='$.2s')
    fig.update_yaxes(title_text="Retention", row=1, col=2, tickformat='$.2s')
    fig.update_yaxes(title_text="Coverage Efficiency (%)", row=2, col=1)
    
    fig.show()
    
    # Print optimization results
    print("\nOptimization Results:")
    print("="*70)
    print(results_df.to_string(index=False))
    
    print("\nRecommended Attachment Points (based on loss distribution):")
    print("-"*70)
    for p, value in attachment_points.items():
        print(f"{p:<20} ${value:,.0f}")
    
    # Find best configuration
    best_idx = results_df['Efficiency'].idxmax()
    print(f"\nBest Configuration: {results_df.loc[best_idx, 'Configuration']}")
    print(f"Coverage Efficiency: {results_df.loc[best_idx, 'Efficiency']*100:.1f}%")
    print(f"Annual Premium: ${results_df.loc[best_idx, 'Premium']:,.0f}")

# Run optimization
optimize_layer_attachments(target_retention_ratio=0.2, n_scenarios=1000)


Optimization Results:
Configuration       Premium  Avg Recovery  Avg Retention  Max Retention  95% VaR Retention  Loss Ratio  Efficiency
 Conservative 245000.000000  25000.000000   1.659839e+06   2.607799e+07       7.132577e+06    0.102041    0.014838
     Balanced 335000.000000  26077.986876   1.658761e+06   2.500000e+07       7.132577e+06    0.077845    0.015478
   Aggressive 448000.000000  49000.000000   1.635839e+06   2.458353e+07       7.458280e+06    0.109375    0.029083
      Optimal 198055.521662  20328.881072   1.664510e+06   2.458353e+07       7.301757e+06    0.102642    0.012066

Recommended Attachment Points (based on loss distribution):
----------------------------------------------------------------------
50th percentile      $388,103
75th percentile      $1,793,643
90th percentile      $4,249,859
95th percentile      $7,481,170
99th percentile      $16,470,025
99.5th percentile    $20,328,881

Best Configuration: Aggressive
Coverage Efficiency: 2.9%
Annual Premium: $448

## 3. Premium Sensitivity Analysis

In [4]:
def premium_sensitivity_analysis():
    """Analyze sensitivity to premium rates."""
    
    # Base configuration
    base_attachments = [0, 5_000_000, 25_000_000]
    base_limits = [5_000_000, 20_000_000, 25_000_000]
    base_rates = [0.015, 0.008, 0.004]
    
    # Generate losses
    loss_generator = ManufacturingLossGenerator(
        attritional_params={
            'base_frequency': 5.0,
            'severity_mean': 50_000,
            'severity_cv': 0.8
        },
        large_params={
            'base_frequency': 0.5,
            'severity_mean': 2_000_000,
            'severity_cv': 1.2
        },
        catastrophic_params={
            'base_frequency': 0.02,
            'severity_xm': 10_000_000,
            'severity_alpha': 2.5
        },
        seed=42
    )
    
    annual_losses = []
    for _ in range(1000):
        events, stats = loss_generator.generate_losses(duration=1.0, revenue=10_000_000)
        annual_losses.append(stats['total_amount'])
    
    # Test different rate multipliers
    multipliers = np.linspace(0.5, 2.0, 16)
    results = []
    
    for mult in multipliers:
        adjusted_rates = [r * mult for r in base_rates]
        
        layers = [
            EnhancedInsuranceLayer(base_attachments[i], base_limits[i], adjusted_rates[i])
            for i in range(3)
        ]
        program = InsuranceProgram(layers)
        
        premium = program.calculate_annual_premium()
        recoveries = []
        for loss in annual_losses:
            recovery_details = program.process_claim(loss)
            recoveries.append(recovery_details['insurance_recovery'])
        
        results.append({
            'multiplier': mult,
            'premium': premium,
            'avg_recovery': np.mean(recoveries),
            'loss_ratio': np.mean(recoveries) / premium if premium > 0 else 0,
            'net_benefit': np.mean(recoveries) - premium
        })
    
    sensitivity_df = pd.DataFrame(results)
    
    # Create visualization
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Premium vs Recovery',
            'Loss Ratio',
            'Net Benefit',
            'Rate Multiplier Impact'
        ),
        specs=[
            [{'type': 'xy'}, {'type': 'xy'}],
            [{'type': 'bar'}, {'type': 'table'}]
        ]
    )
    
    # Premium vs Recovery
    fig.add_trace(
        go.Scatter(
            x=sensitivity_df['multiplier'],
            y=sensitivity_df['premium'],
            mode='lines',
            name='Premium',
            line=dict(color=WSJ_COLORS['blue'], width=2)
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=sensitivity_df['multiplier'],
            y=sensitivity_df['avg_recovery'],
            mode='lines',
            name='Avg Recovery',
            line=dict(color=WSJ_COLORS['orange'], width=2, dash='dash')
        ),
        row=1, col=1
    )
    
    # Loss Ratio
    fig.add_trace(
        go.Scatter(
            x=sensitivity_df['multiplier'],
            y=sensitivity_df['loss_ratio'] * 100,
            mode='lines+markers',
            name='Loss Ratio',
            line=dict(color=WSJ_COLORS['red'], width=2)
        ),
        row=1, col=2
    )
    
    # Add 100% reference line
    fig.add_hline(
        y=100,
        line_dash="dash",
        line_color="gray",
        annotation_text="Break-even",
        row=1, col=2
    )
    
    # Net Benefit
    fig.add_trace(
        go.Bar(
            x=sensitivity_df['multiplier'],
            y=sensitivity_df['net_benefit'],
            marker_color=np.where(sensitivity_df['net_benefit'] > 0, 
                                 WSJ_COLORS['green'], WSJ_COLORS['red'])
        ),
        row=2, col=1
    )
    
    # Rate impact table
    selected_mults = [0.5, 0.75, 1.0, 1.25, 1.5]
    selected_data = sensitivity_df[sensitivity_df['multiplier'].isin(selected_mults)]
    
    fig.add_trace(
        go.Table(
            header=dict(
                values=['Multiplier', 'Premium', 'Recovery', 'Loss Ratio', 'Net Benefit'],
                fill_color=WSJ_COLORS['light_gray'],
                align='left'
            ),
            cells=dict(
                values=[
                    [f'{x:.2f}x' for x in selected_data['multiplier']],
                    ['${:,.0f}'.format(x) for x in selected_data['premium']],
                    ['${:,.0f}'.format(x) for x in selected_data['avg_recovery']],
                    ['{:.0f}%'.format(x*100) for x in selected_data['loss_ratio']],
                    ['${:,.0f}'.format(x) for x in selected_data['net_benefit']]
                ],
                align='left'
            )
        ),
        row=2, col=2
    )
    
    # Update layout
    fig.update_layout(
        height=800,
        showlegend=True,
        title_text="Premium Rate Sensitivity Analysis",
        template='plotly_white'
    )
    
    fig.update_xaxes(title_text="Rate Multiplier", row=1, col=1)
    fig.update_xaxes(title_text="Rate Multiplier", row=1, col=2)
    fig.update_xaxes(title_text="Rate Multiplier", row=2, col=1)
    
    fig.update_yaxes(title_text="Amount", row=1, col=1, tickformat='$.2s')
    fig.update_yaxes(title_text="Loss Ratio (%)", row=1, col=2)
    fig.update_yaxes(title_text="Net Benefit", row=2, col=1, tickformat='$.2s')
    
    fig.show()
    
    # Find optimal multiplier
    optimal_idx = sensitivity_df['net_benefit'].idxmax()
    optimal_mult = sensitivity_df.loc[optimal_idx, 'multiplier']
    
    print("\nSensitivity Analysis Summary:")
    print("="*70)
    print(f"Base annual premium: ${base_rates[0]*base_limits[0] + base_rates[1]*base_limits[1] + base_rates[2]*base_limits[2]:,.0f}")
    print(f"Average annual loss: ${np.mean(annual_losses):,.0f}")
    print(f"\nOptimal rate multiplier: {optimal_mult:.2f}x")
    print(f"Optimal premium: ${sensitivity_df.loc[optimal_idx, 'premium']:,.0f}")
    print(f"Expected recovery: ${sensitivity_df.loc[optimal_idx, 'avg_recovery']:,.0f}")
    print(f"Net benefit: ${sensitivity_df.loc[optimal_idx, 'net_benefit']:,.0f}")
    print(f"Loss ratio at optimum: {sensitivity_df.loc[optimal_idx, 'loss_ratio']*100:.1f}%")

# Run sensitivity analysis
premium_sensitivity_analysis()


Sensitivity Analysis Summary:
Base annual premium: $335,000
Average annual loss: $1,684,839

Optimal rate multiplier: 0.50x
Optimal premium: $167,500
Expected recovery: $26,078
Net benefit: $-141,422
Loss ratio at optimum: 15.6%


## Summary

This notebook provides comprehensive tools for insurance layer optimization:

1. **Interactive Layer Design**: Visual design and testing of multi-layer structures
2. **Attachment Point Optimization**: Data-driven selection of optimal attachment points
3. **Premium Sensitivity**: Understanding the trade-offs between premium and coverage

Key insights:
- Optimal attachment points align with loss distribution percentiles
- Layer efficiency depends on both structure and pricing
- Premium sensitivity shows diminishing returns beyond certain rates
- Multi-layer programs provide better risk transfer than single large layers